In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import json
import h2o

from datetime import datetime

from utils import get_data, Config, clean_url, make_clickable, add_columns

from tree import make_tree

In [156]:
# Load all data (except unknown/manual testing)
# Original Desktop Runs+Re-Runs were performed between 2024-02-05 and 2024-02-12
# Additional browsers + more than one response_id per parsing URL was run between 2024-02-16 and TBD
# Mobile runs performed between 2024-02-28 and TBD
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown' and "Response".resp_type = 'basic' and "Result".created_at::date = '2024-03-04' and "Browser".os = 'Ubuntu 22.04'; -- AND "Result".created_at < '2024-02-19';
"""
df = get_data(Config(), initial_data)
df = add_columns(df)

Connecting to the PostgreSQL database...
Connection successful


In [157]:
df["raw_header"] = df["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str)
def create_test_id(row):
    return f'{row["test_name"]}_{row["relation_info"]}_{row["org_scheme"]}_{row["org_host"]}_{row["resp_scheme"]}_{row["resp_host"]}_{row["response_id"]}_{row["resp_type"]}'

df["browser_id"] = df["browser_id"].astype("category")
# Takes a while (500s+) (might be faster to already do it with postgres but not too important)
df["test_id"] = df.apply(create_test_id, axis=1)
df["test_id"] = df["test_id"].astype("category")

In [158]:
responses = """
SELECT * from "Response";
"""
responses = get_data(Config(), responses)

Connecting to the PostgreSQL database...
Connection successful


# Overview

In [159]:
# Both basic + parsing mode
# Each test should have results 5+ times (such that we can better reason about noise + majority voting makes sense)
# Basic was executed 20x+ on Linux, 5x on Mac
# Parsing was executed 5+ on Linux, 2x on Mac (currently executing a third time)
# (In addition, a couple of "repeat"-mode runs were performed in the beginning
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"], observed=True)["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,selenium,47,chrome,122,Ubuntu 22.04,headless-new,65984
1,selenium,48,chrome,120,Ubuntu 22.04,headless-new,52640
2,selenium,42,chrome,121,Ubuntu 22.04,headless-new,52632
3,selenium,44,edge,121,Ubuntu 22.04,headless-new,52608
4,selenium,45,brave,v1.62.156 (121.0.6167.139),Ubuntu 22.04,headless-new,52592
5,selenium,43,firefox,122,Ubuntu 22.04,headless,52424
6,selenium,49,firefox,121,Ubuntu 22.04,headless,52376


# Error + Timeout Analysis
- Only timeout exist, no other systematic failures anymore
- Systematic timeout: TAO in Safari -> fixed
- Couple of random timeouts in other tests
  - More basic tests than parsing tests (even though less exist)
  - Code 302 more often than others
  - referrer-iframe, oac_*, framing_iframe most often

In [160]:
# Timeout (2), Fail (1), Not-run (3)
filtered_df = df.groupby(["browser"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["browser", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["browser"])["test_status"].value_counts().to_frame())

count
browser                                       test_status       
chrome Ubuntu 22.04 121 selenium headless-new 0            52629
                                              2                3
chrome Ubuntu 22.04 122 selenium headless-new 0            65982
                                              2                2
firefox Ubuntu 22.04 121 selenium headless    0            52356
                                              2               20
firefox Ubuntu 22.04 122 selenium headless    0            52412
                                              2               12

In [161]:
# Teststatus == 2 (timeout)

# Rare timeouts in all browsers; Safari some more than others
display(df[df["test_status"] == 2]["browser"].value_counts().to_frame())

# Mostly 302 redirects for basic (oac_iframe, oac_window.open), referrer_iframe 
# perfAPI_img/TAO (only Safari, fixed!)
# Others quite rare
with pd.option_context("display.max_rows", 80):
    display(df.loc[(df["test_status"] == 2)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
firefox Ubuntu 22.04 121 selenium headless,20
firefox Ubuntu 22.04 122 selenium headless,12
chrome Ubuntu 22.04 121 selenium headless-new,3
chrome Ubuntu 22.04 122 selenium headless-new,2


count
resp_type status_code test_name               browser                                             
basic     200         fetch_GET               firefox Ubuntu 22.04 121 selenium headless        11
                                              firefox Ubuntu 22.04 122 selenium headless         6
                      referrer_iframe         firefox Ubuntu 22.04 121 selenium headless         5
                      fetch_TEST              firefox Ubuntu 22.04 121 selenium headless         4
                      referrer_iframe         firefox Ubuntu 22.04 122 selenium headless         4
                      fullscreen_iframe       chrome Ubuntu 22.04 121 selenium headless-new      3
                      fetch_TEST              firefox Ubuntu 22.04 122 selenium headless         2
          302         script_execution_iframe chrome Ubuntu 22.04 122 selenium headless-new      2

In [162]:
# Teststatus == 3 (not-run)

# Rare timeouts in all browsers; Safari some more than others
display(df[df["test_status"] == 3]["browser"].value_counts().to_frame())

# Mostly 302 redirects for basic (oac_iframe, oac_window.open), referrer_iframe 
# perfAPI_img/TAO (only Safari, fixed!)
# Others quite rare
display(df.loc[(df["test_status"] == 3)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,


,,,,count
resp_type,status_code,test_name,browser,


# General Test Statistics

In [163]:
df[["browser", "resp_type", "test_name", "relation_info", "response_id", "org_scheme", "org_host", "resp_scheme", "resp_host"]].drop_duplicates().groupby(["resp_type", "browser"]).count()

test_name  \
resp_type browser                                                         
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      10456   
          chrome Ubuntu 22.04 120 selenium headless-new           10456   
          chrome Ubuntu 22.04 121 selenium headless-new           10456   
          chrome Ubuntu 22.04 122 selenium headless-new           10456   
          edge Ubuntu 22.04 121 selenium headless-new             10456   
          firefox Ubuntu 22.04 121 selenium headless              10456   
          firefox Ubuntu 22.04 122 selenium headless              10456   

                                                              relation_info  \
resp_type browser                                                             
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...          10456   
          chrome Ubuntu 22.04 120 selenium headless-new               10456   
          chrome Ubuntu 22.04 121 selenium headless-new               10456   
          chrome Ubuntu 22.04 122 selenium headless-new               10456   
          edge Ubuntu 22.04 121 selenium headless-new                 10456   
          firefox Ubuntu 22.04 121 selenium headless                  10456   
          firefox Ubuntu 22.04 122 selenium headless                  10456   

                                                              response_id  \
resp_type browser                                                           
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...        10456   
          chrome Ubuntu 22.04 120 selenium headless-new             10456   
          chrome Ubuntu 22.04 121 selenium headless-new             10456   
          chrome Ubuntu 22.04 122 selenium headless-new             10456   
          edge Ubuntu 22.04 121 selenium headless-new               10456   
          firefox Ubuntu 22.04 121 selenium headless                10456   
          firefox Ubuntu 22.04 122 selenium headless                10456   

                                                              org_scheme  \
resp_type browser                                                          
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...       10456   
          chrome Ubuntu 22.04 120 selenium headless-new            10456   
          chrome Ubuntu 22.04 121 selenium headless-new            10456   
          chrome Ubuntu 22.04 122 selenium headless-new            10456   
          edge Ubuntu 22.04 121 selenium headless-new              10456   
          firefox Ubuntu 22.04 121 selenium headless               10456   
          firefox Ubuntu 22.04 122 selenium headless               10456   

                                                              org_host  \
resp_type browser                                                        
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     10456   
          chrome Ubuntu 22.04 120 selenium headless-new          10456   
          chrome Ubuntu 22.04 121 selenium headless-new          10456   
          chrome Ubuntu 22.04 122 selenium headless-new          10456   
          edge Ubuntu 22.04 121 selenium headless-new            10456   
          firefox Ubuntu 22.04 121 selenium headless             10456   
          firefox Ubuntu 22.04 122 selenium headless             10456   

                                                              resp_scheme  \
resp_type browser                                                           
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...        10456   
          chrome Ubuntu 22.04 120 selenium headless-new             10456   
          chrome Ubuntu 22.04 121 selenium headless-new             10456   
          chrome Ubuntu 22.04 122 selenium headless-new             10456   
          edge Ubuntu 22.04 121 selenium headless-new               10456   
          firefox Ubuntu 22.04 121 selenium headless                10456   
          fi

In [164]:
168774 + 10456

179230

In [165]:
# Some browsers only used the more response_ids per page mode, thus these numbers don't say much
df.groupby("browser")["clean_url"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    364
chrome Ubuntu 22.04 120 selenium headless-new                          364
chrome Ubuntu 22.04 121 selenium headless-new                          364
chrome Ubuntu 22.04 122 selenium headless-new                          364
edge Ubuntu 22.04 121 selenium headless-new                            364
firefox Ubuntu 22.04 121 selenium headless                             364
firefox Ubuntu 22.04 122 selenium headless                             364
Name: clean_url, dtype: int64

In [166]:
# Number of response_ids tested (should be 44415)
# Many response_ids have more than one test (e.g., both iframe and window.open) thus this is not enough to show that all tests have results!
df.groupby("browser")["response_id"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    189
chrome Ubuntu 22.04 120 selenium headless-new                          189
chrome Ubuntu 22.04 121 selenium headless-new                          189
chrome Ubuntu 22.04 122 selenium headless-new                          189
edge Ubuntu 22.04 121 selenium headless-new                            189
firefox Ubuntu 22.04 121 selenium headless                             189
firefox Ubuntu 22.04 122 selenium headless                             189
Name: response_id, dtype: int64

In [167]:
df.loc[df["test_status"] == 0].groupby("browser")["test_id"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    10456
chrome Ubuntu 22.04 120 selenium headless-new                          10456
chrome Ubuntu 22.04 121 selenium headless-new                          10456
chrome Ubuntu 22.04 122 selenium headless-new                          10456
edge Ubuntu 22.04 121 selenium headless-new                            10456
firefox Ubuntu 22.04 121 selenium headless                             10456
firefox Ubuntu 22.04 122 selenium headless                             10456
Name: test_id, dtype: int64

In [168]:
# Make sure that all responses actually have responses
# (create_repeat.py assumes that at least one browser collected the results)

r1 = df.groupby("label")["response_id"].nunique().sort_values()
r2 = responses.loc[responses["resp_type"] != "debug"].groupby("label")["id"].nunique().sort_values()

pd.concat([r1, r2], axis=1)

,response_id,id
label,,
CORS,6.0,6
CSPvsXFO,7.0,1447
COOP,11.0,2667
COEP,11.0,2648
CORP,11.0,3368
XFO,11.0,4607
TAO,13.0,1614
HSTS,14.0,3338
OAC,14.0,1713


In [169]:
# Number of response_ids for each "group": resp_type, test_name, relation_info
df.groupby(["resp_type", "test_name", "relation_info"])["response_id"].nunique().to_frame()

response_id
resp_type test_name                     relation_info              
basic     accesswindow_direct           direct                   11
          fetch_GET                     credentials               6
                                        custom_headers            6
                                        simple                    6
          fetch_TEST                    custom_method             6
          framing_embed                 direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_iframe                direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_object                direct                   35
                                        nested                   35
                                        sandbox                  35
          fullscreen_iframe             child                    24
                                        child_allow              24
                                        child_sandbox            24
                                        direct                   24
          imgloading_iframe             direct                   17
          oac_iframe                    direct                   14
                                        sandbox                  14
          oac_window.open               window.open              14
          perfAPI_img                   direct                   13
          referrer_iframe               iframe                   16
                                        window.open              16
          script_execution_iframe       direct                   17
                                        sandbox                  17
          subresourceloadingCOEP_img    direct                   11
                                        sandbox                  11
          subresourceloadingCORP_img    direct                   11
                                        nested                   11
                                        sandbox                  11
          subresourceloadingCORP_object direct                   11
          upgradeHSTS_direct            direct                   14
          upgradeHSTS_subdomain         subdomain                14

# Stability (Results after 2x with only one response_id per parsing URL test)
- Basic tests:
    - 21/(10456x5) tests with different outcomes!
    - 13x firefox subresourceloadingCOEP_img (random behavior bug)
    - 6x timeout vs message 4(x edge, 2x brave)
    - 1x brave referrer different outcome
    - 1x safari COEP different outcome?
- Parsing tests:
    - 755/(168774x5) tests with different outcomes!
    - quite a lot! last time we only had 1 single test with a different outcome? (We used different versions of the browsers)
    - 3x accesswindow_direct (null vs accessible), 1x fullscreen_iframe (timeout vs message), 1x img_loading_iframe (timeout vs message)
    - All others (700+) upgradeHSTS_subdomain and upgradeHSTS_direct
        - Prevalence: `brave ~= edge > chrome >> firefox >> safari`
        - Hints at systematic issue: maybe the reset did not work or something similar?
        - Mostly timeout vs redirect=false; however some are also timeout vs redirect=false vs redirect=true!
        - Timeout maybe related to the HTTPS upgrade feature in chromium-based browsers which can take some time (>5s?)? (Non-Authoritative-Reason: HttpsUpgrades)
        - Unclear how redirect true vs false could happen? Maybe HSTS clearing did not work?
        - Manual testing always results in redirect=false for all tried clean_urls!

### Update: multiple response_ids per parsing URL test
- basic (123+)
    - mostly firefox subresourceloadingCOEP_img (random behavior bug)
    - some rare instances of other tests (timeouts)
- parsing (5000+)
    - fetch_* timeout issue in firefox (fixed)
    - subresourceloadingCOEP_img and imgloading_iframe timeout issues (other browsers) (probably fixed)
    - upgradeHSTS_subdomain + upgradeHSTS_direct
    - rare instances of other tests
- mostly timeouts?: Firefox fetch, other browsers subresourceloadingCOEP_img
- maybe we have too many tests per page/or the timeout is not high enough -> decreased the number of tests for some feature groups (now it seems better)
- (OT: if running `desktop_selenium.py` in the repeat mode, one has to increate `--timeout_task` (15*100 = 1500s > 1000) or decrease `--max_urls_until_restart` as we ran the repeat tests with a 3xTIMEOUT)

In [170]:
# Remove all tests that timed out (test_status == 0), each test has at least one result that did not time out in each browser
df_org = df
df = df.loc[df["test_status"] == 0]

In [171]:
details = True
for group_name, resp_type_group in df.groupby("resp_type"):
    #if group_name == "basic":
    #    continue
    print(group_name)
    stab = resp_type_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    with pd.option_context("display.max_rows", 100):
        display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()

    if not details:
        continue
    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        # TODO: shortcut, only show the first result for each test_name/browser; results are usually always the same!
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "browser"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)
        # input("Continue!")

basic
569 tests have different outcomes!


,count
browser,
chrome Ubuntu 22.04 122 selenium headless-new,93
edge Ubuntu 22.04 121 selenium headless-new,89
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,87
chrome Ubuntu 22.04 120 selenium headless-new,84
chrome Ubuntu 22.04 121 selenium headless-new,83
firefox Ubuntu 22.04 121 selenium headless,73
firefox Ubuntu 22.04 122 selenium headless,60


count
(test_name, )              (browser, )                                              
subresourceloadingCOEP_img firefox Ubuntu 22.04 121 selenium headless             69
                           firefox Ubuntu 22.04 122 selenium headless             52
                           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     47
                           chrome Ubuntu 22.04 120 selenium headless-new          45
                           chrome Ubuntu 22.04 122 selenium headless-new          42
                           edge Ubuntu 22.04 121 selenium headless-new            39
                           chrome Ubuntu 22.04 121 selenium headless-new          37
accesswindow_direct        chrome Ubuntu 22.04 121 selenium headless-new          32
                           chrome Ubuntu 22.04 122 selenium headless-new          23
                           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     21
                           chrome Ubuntu 22.04 120 selenium headless-new          20
fetch_GET                  edge Ubuntu 22.04 121 selenium headless-new            17
accesswindow_direct        edge Ubuntu 22.04 121 selenium headless-new            16
fetch_GET                  chrome Ubuntu 22.04 122 selenium headless-new          14
                           chrome Ubuntu 22.04 120 selenium headless-new          10
                           chrome Ubuntu 22.04 121 selenium headless-new          10
                           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      8
referrer_iframe            brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      6
script_execution_iframe    chrome Ubuntu 22.04 120 selenium headless-new           5
                           edge Ubuntu 22.04 121 selenium headless-new             5
                           chrome Ubuntu 22.04 122 selenium headless-new           5
referrer_iframe            edge Ubuntu 22.04 121 selenium headless-new             5
fetch_TEST                 edge Ubuntu 22.04 121 selenium headless-new             4
accesswindow_direct        firefox Ubuntu 22.04 122 selenium headless              4
oac_window.open            brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      4
referrer_iframe            chrome Ubuntu 22.04 122 selenium headless-new           4
fetch_GET                  firefox Ubuntu 22.04 122 selenium headless              3
referrer_iframe            chrome Ubuntu 22.04 121 selenium headless-new           2
accesswindow_direct        firefox Ubuntu 22.04 121 selenium headless              2
upgradeHSTS_subdomain      chrome Ubuntu 22.04 120 selenium headless-new           2
fetch_TEST                 chrome Ubuntu 22.04 122 selenium headless-new           2
fetch_GET                  firefox Ubuntu 22.04 121 selenium headless              1
script_execution_iframe    firefox Ubuntu 22.04 122 selenium headless              1
oac_window.open            chrome Ubuntu 22.04 120 selenium headless-new           1
referrer_iframe            chrome Ubuntu 22.04 120 selenium headless-new           1
oac_window.open            edge Ubuntu 22.04 121 selenium headless-new             1
perfAPI_img                firefox Ubuntu 22.04 121 selenium headless              1
subresourceloadingCORP_img chrome Ubuntu 22.04 122 selenium headless-new           1
upgradeHSTS_direct         chrome Ubuntu 22.04 121 selenium headless-new           1
                           edge Ubuntu 22.04 121 selenium headless-new             1
                           chrome Ubuntu 22.04 122 selenium headless-new           1
upgradeHSTS_subdomain      brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                           chrome Ubuntu 22.04 121 selenium headless-new           1
                           chrome Ubuntu 22.04 122 selenium headless-new           1
                           edge Ubuntu 22.04 121 selenium headless-new             1

accesswindow_direct [80, 81]


,,count
outcome_str,browser,
{'window.open.opener': 'null'},brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,8
"{'window.open.opener': 'object ""[object Window]""'}",brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,2


...

,,count
outcome_str,browser,
{'response.redirected': False},chrome Ubuntu 22.04 122 selenium headless-new,6
Fetch timed out,chrome Ubuntu 22.04 122 selenium headless-new,1


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
11207,chrome Ubuntu 22.04 122 selenium headless-new,{'response.redirected': False},"[['Strict-Transport-Security', 'includeSubDomains']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=135&last_id=135&scheme=http
376215,chrome Ubuntu 22.04 122 selenium headless-new,Fetch timed out,"[['Strict-Transport-Security', 'includeSubDomains']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=135&last_id=135&scheme=http


upgradeHSTS_subdomain [134]


,,count
outcome_str,browser,
{'response.redirected': True},edge Ubuntu 22.04 121 selenium headless-new,4
Fetch timed out,edge Ubuntu 22.04 121 selenium headless-new,1


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
88141,edge Ubuntu 22.04 121 selenium headless-new,{'response.redirected': True},"[['Strict-Transport-Security', 'max-age=20; includeSubDomains']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=134&last_id=134&scheme=http
376205,edge Ubuntu 22.04 121 selenium headless-new,Fetch timed out,"[['Strict-Transport-Security', 'max-age=20; includeSubDomains']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=134&last_id=134&scheme=http


## More stability?!
- How many outcomes did we collect per test in the different browsers? Also shows tests with only one outcome
- How many of them are different?

In [172]:
# How many outcomes did we collect for each test?!
# Should be between 5 and 20+ (less than 5 is concerning as we cannot do proper majority voting in such cases)

final_df = pd.DataFrame()

for group, d in df.groupby("browser"):
    # Calculate counts and unique counts
    aggs = ["count"] # ["count", "nunique"]
    res = d.groupby(["test_id"], observed=True)["outcome_str"].agg(aggs)
    # Get value counts for each combination of counts and unique counts
    counts = res.value_counts().to_frame()    
    counts = counts.rename(columns={"count": group})
    display(counts)
    final_df = pd.concat([final_df, counts], axis=1)

display(final_df)

,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new
count,
5,10144
6,312


,chrome Ubuntu 22.04 120 selenium headless-new
count,
5,10096
6,360


,chrome Ubuntu 22.04 121 selenium headless-new
count,
5,10136
6,291
7,29


,chrome Ubuntu 22.04 122 selenium headless-new
count,
7,6118
5,2872
6,1466


,edge Ubuntu 22.04 121 selenium headless-new
count,
5,10128
6,328


,firefox Ubuntu 22.04 121 selenium headless
count,
5,10409
7,29
6,18


,firefox Ubuntu 22.04 122 selenium headless
count,
5,10360
6,60
7,36


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless
count,,,,,,,
5,10144.0,10096.0,10136,2872,10128.0,10409,10360
6,312.0,360.0,291,1466,328.0,18,60
7,NaN,NaN,29,6118,NaN,29,36


In [173]:
# Which tests have how many outcomes?

grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].count().reset_index()

with pd.option_context("display.max_rows", 256):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["outcome_str"].agg(["mean", "min", "max", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0])

,mean,min,max,count,sum
count,126.000000,126.000000,126.000000,126.000000,126.000000
mean,5.241528,5.119048,5.714286,580.888889,3025.547619
std,0.556292,0.430946,0.778276,591.006494,3075.502536
min,5.000000,5.000000,5.000000,28.000000,140.000000
25%,5.000000,5.000000,5.000000,176.000000,896.000000
50%,5.028571,5.000000,6.000000,320.000000,1760.000000
75%,5.061581,5.000000,6.000000,544.000000,2748.000000
max,7.000000,7.000000,7.000000,1680.000000,10344.000000


mean  \
browser                                            resp_type test_name                                 
chrome Ubuntu 22.04 122 selenium headless-new      basic     subresourceloadingCORP_img     7.000000   
                                                             subresourceloadingCORP_object  7.000000   
                                                             upgradeHSTS_direct             7.000000   
                                                             subresourceloadingCOEP_img     7.000000   
                                                             upgradeHSTS_subdomain          7.000000   
                                                             script_execution_iframe        6.966912   
                                                             imgloading_iframe              6.941176   
                                                             accesswindow_direct            6.909091   
                                                             fetch_TEST                     6.500000   
                                                             fetch_GET                      6.500000   
                                                             referrer_iframe                6.500000   
                                                             framing_embed                  6.157143   
                                                             framing_iframe                 6.157143   
                                                             framing_object                 6.157143   
                                                             fullscreen_iframe              6.000000   
                                                             perfAPI_img                    6.000000   
                                                             oac_iframe                     5.928571   
                                                             oac_window.open                5.928571   
firefox Ubuntu 22.04 122 selenium headless         basic     fetch_TEST                     5.229167   
                                                             fetch_GET                      5.229167   
firefox Ubuntu 22.04 121 selenium headless         basic     fetch_GET                      5.128472   
                                                             fetch_TEST                     5.125000   
chrome Ubuntu 22.04 120 selenium headless-new      basic     oac_iframe                     5.107143   
                                                             oac_window.open                5.107143   
edge Ubuntu 22.04 121 selenium headless-new        basic     accesswindow_direct            5.090909   
chrome Ubuntu 22.04 120 selenium headless-new      basic     accesswindow_direct            5.090909   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     accesswindow_direct            5.090909   
chrome Ubuntu 22.04 121 selenium headless-new      basic     fullscreen_iframe              5.081380   
edge Ubuntu 22.04 121 selenium headless-new        basic     oac_window.open                5.071429   
                                                             oac_iframe                     5.071429   
chrome Ubuntu 22.04 120 selenium headless-new      basic     referrer_iframe                5.062500   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     referrer_iframe                5.062500   
                                                             imgloading_iframe              5.058824   
chrome Ubuntu 22.04 121 selenium headless-new      basic     script_execution_iframe        5.058824   
chrome Ubuntu 22.04 120 selenium headless-new      basic     imgloading_iframe              5.058824   
chrome Ubuntu 22.04 121 selenium headless-new      basic     imgloading_iframe              5.058824   
firefox Ubuntu 22.04 122 selenium headless         basic     referrer_iframe                5.054688   
firefox Ubuntu 22.04 121 selenium headle

In [174]:
# Different outcomes (in percentage for a test group (test_name)
# (Does not take into account how often each test was executed)
grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].nunique().reset_index()

grouped["diff_outcome"] = grouped["outcome_str"] > 1
with pd.option_context("display.max_rows", 100):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["diff_outcome"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0])

,mean,count,sum
count,126.000000,126.000000,126.000000
mean,0.018345,580.888889,4.515873
std,0.040207,591.006494,11.805581
min,0.000000,28.000000,0.000000
25%,0.000000,176.000000,0.000000
50%,0.000000,320.000000,0.000000
75%,0.010254,544.000000,1.000000
max,0.196023,1680.000000,69.000000


,,,mean,count,sum
browser,resp_type,test_name,,,
firefox Ubuntu 22.04 121 selenium headless,basic,subresourceloadingCOEP_img,0.196023,352,69
chrome Ubuntu 22.04 121 selenium headless-new,basic,accesswindow_direct,0.181818,176,32
firefox Ubuntu 22.04 122 selenium headless,basic,subresourceloadingCOEP_img,0.147727,352,52
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,basic,subresourceloadingCOEP_img,0.133523,352,47
chrome Ubuntu 22.04 122 selenium headless-new,basic,accesswindow_direct,0.130682,176,23
chrome Ubuntu 22.04 120 selenium headless-new,basic,subresourceloadingCOEP_img,0.127841,352,45
chrome Ubuntu 22.04 122 selenium headless-new,basic,subresourceloadingCOEP_img,0.119318,352,42
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,basic,accesswindow_direct,0.119318,176,21
chrome Ubuntu 22.04 120 selenium headless-new,basic,accesswindow_direct,0.113636,176,20


In [175]:
# Run the test groups with many different outcomes again?
# Groups with non-negligble different outcomes are subresourceloadingCOEP_img, upgradeHSTS_subdomain, upgradeHSTS_direct, imgloading_iframe, fetch_TEST, fetch_GET
test_files_to_repeat =  ["fetch-cors.sub.html", "subresource-loading-coep.sub.html", "subresource-loading-csp.sub.html", "upgrade-hsts.sub.html"]

# TODO: implement utils.py/desktop_selenium mode to only rerun these files?!
# For now, simply run everything again to get some more results for stability?!

In [176]:
# Majority voting for the tests with multiple outcomes
df.sample(100).groupby(["test_name", "relation_info", "browser", "org_host", "resp_scheme", "org_origin", "resp_origin", "response_id"])["outcome_str"].agg([pd.Series.mode, "nunique", "count"]).reset_index().sort_values("count")

,test_name,relation_info,browser,org_host,resp_scheme,org_origin,resp_origin,response_id,mode,nunique,count
88,referrer_iframe,window.open,chrome Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://sub.sub.headers.websec.saarland,197,document.referrer: https://sub.sub.headers.web...,1,1
89,referrer_iframe,window.open,firefox Ubuntu 22.04 122 selenium headless,sub.headers.websec.saarland,http,https://sub.headers.websec.saarland,http://headers.webappsec.eu,192,document.referrer: full_url,1,1
90,script_execution_iframe,direct,chrome Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,http,http://sub.headers.websec.saarland,http://sub.headers.websec.saarland,104,message send,1,1
91,script_execution_iframe,direct,chrome Ubuntu 22.04 122 selenium headless-new,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.websec.saarland,101,message send,1,1
92,script_execution_iframe,sandbox,edge Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://headers.websec.saarland,103,message send,1,1
...,...,...,...,...,...,...,...,...,...,...,...
15,framing_embed,sandbox,chrome Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,http,https://sub.headers.websec.saarland,http://headers.webappsec.eu,33,message timeout,1,1
97,subresourceloadingCORP_img,sandbox,edge Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,http,http://sub.headers.websec.saarland,http://sub.headers.websec.saarland,46,error,1,1
98,subresourceloadingCORP_object,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,http,https://sub.headers.websec.saarland,http://sub.headers.websec.saarland,55,error,1,1
99,subresourceloadingCORP_object,direct,firefox Ubuntu 22.04 121 selenium headless,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.webappsec.eu,55,error,1,1


# Browser differences!
- First perform majority voting
- Then diff the browsers

In [177]:
df = df_org
df = df.loc[df["test_status"] == 0]

In [178]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, use majority voting! (only works reliably if there a enough repetitions of the tests runs, >= 5)

print("Original data entries", len(df))

# Majority voting (pd.Series.mode returns the most frequest item) (quite slow)

df = df.groupby(["test_id", "browser"], observed=True)["outcome_str"].agg(pd.Series.mode).reset_index()
print("Only one row for each test (per browser; majority voting)", len(df))

Original data entries 381219
Only one row for each test (per browser; majority voting) 73192


In [179]:
# Merge back additional required properties!
# ["name", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "resp_type", "label", 'org_origin', 'resp_origin', 'status_code', 'raw_header']

df = df.merge(df_stab.drop_duplicates(subset=["test_id", "browser"]), on=["test_id", "browser"], how="left", suffixes=["", "_ignore"])


In [180]:
# If several values occur the same, the mode is an np.ndarray and not a string!
# What to do in such cases? If we have more repetitions the issue should probably go away automatically?
# If we have exactly 5 repetitions for each test, the issue cannot exist! (in the debug phase we sometimes have more or less than 5 repetitions and the issue can exist)
print("Tests with the same mode (highest frequency outcome):")
display(df.loc[df['outcome_str'].apply(lambda x: isinstance(x, np.ndarray))].groupby(["browser", "resp_type", "test_name"])["outcome_str"].count())

# In the rare cases, where it actually matters, we can just take the first? (this might be noise in the generated trees)
df['outcome_str'] = df['outcome_str'].apply(lambda x: x[0] if isinstance(x, np.ndarray) else x)

Tests with the same mode (highest frequency outcome):


Series([], Name: outcome_str, dtype: int64)

In [181]:
# Most tests have exactly one outcome, quite some have two, couple have three!
res = df.groupby(["resp_type", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "test_id"], observed=True)
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

,count
outcome_str,
1,9383
2,1070
3,3


In [182]:
# Basic:
# subresourceloadingCORP_object has different results for 100% of tests
# couple of others have 20%+ differences
# couple have 0%

# Parsing:
# oac_window.open has 99%
# Other test groups have between 0.5 - 7.5% of tests with different outcomes
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["resp_type", "test_name", "relation_info"])["diff"].agg(["count", "sum", "mean"]).sort_values("mean", ascending=False)

count  sum      mean
resp_type test_name                     relation_info                       
basic     oac_window.open               window.open       224  192  0.857143
          oac_iframe                    direct            224  144  0.642857
                                        sandbox           224  144  0.642857
          referrer_iframe               window.open       256  117  0.457031
          fullscreen_iframe             direct            384  142  0.369792
                                        child_allow       384  113  0.294271
          subresourceloadingCOEP_img    direct            176   46  0.261364
          perfAPI_img                   direct            208   52  0.250000
          subresourceloadingCORP_object direct            176   32  0.181818
          accesswindow_direct           direct            176   28  0.159091
          referrer_iframe               iframe            256   18  0.070312
          fullscreen_iframe             child             384   23  0.059896
          imgloading_iframe             direct            272   13  0.047794
          framing_iframe                sandbox           560    8  0.014286
          subresourceloadingCORP_img    direct            176    1  0.005682
          fetch_GET                     custom_headers     96    0  0.000000
                                        credentials        96    0  0.000000
          framing_embed                 direct            560    0  0.000000
          fetch_GET                     simple             96    0  0.000000
          fullscreen_iframe             child_sandbox     384    0  0.000000
          framing_object                direct            560    0  0.000000
                                        nested            560    0  0.000000
                                        sandbox           560    0  0.000000
          framing_embed                 nested            560    0  0.000000
                                        sandbox           560    0  0.000000
          framing_iframe                direct            560    0  0.000000
                                        nested            560    0  0.000000
          fetch_TEST                    custom_method      96    0  0.000000
          script_execution_iframe       direct            272    0  0.000000
                                        sandbox           272    0  0.000000
          subresourceloadingCOEP_img    sandbox           176    0  0.000000
          subresourceloadingCORP_img    nested            176    0  0.000000
                                        sandbox           176    0  0.000000
          upgradeHSTS_direct            direct             28    0  0.000000
          upgradeHSTS_subdomain         subdomain          28    0  0.000000

In [183]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]
tests_with_more_than_one_outcome = df_mult.drop_duplicates(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])


all_tests_should_be =  df_org["test_id"].nunique() * df_org["browser"].nunique() # (num parsing tests + num_basic_tests) * num_browsers (168774+10456)
print(f"All test rows: {len(df)}, should be: {all_tests_should_be}, Rows with more than one outcome: {len(df_mult)}, Tests with more than one outcome: {len(tests_with_more_than_one_outcome)}")

All test rows: 73192, should be: 73192, Rows with more than one outcome: 7511, Tests with more than one outcome: 1073


In [184]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

('accesswindow_direct', 'direct')
1


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"


('framing_iframe', 'sandbox')
1


,browser
outcome_str,
message send,"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
message timeout,"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"


('fullscreen_iframe', 'child')
1


...

,browser
outcome_str,
"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"


('subresourceloadingCORP_img', 'direct')
1


,browser
outcome_str,
error,"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
load,"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"


('subresourceloadingCORP_object', 'direct')
1


,browser
outcome_str,
error,"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
load,"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"


In [185]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['browser'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    unique_outcomes = {}
    group.groupby(["browser"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

('accesswindow_direct', 'direct')


,"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']","['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']"
{'window.open.opener': 'null'},28.0,NaN
"{'window.open.opener': 'object ""[object Window]""'}",NaN,28.0



('framing_iframe', 'sandbox')


,"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
message send,8.0,NaN
message timeout,NaN,8.0



('fullscreen_iframe', 'child')


...

,"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']",['firefox Ubuntu 22.04 121 selenium headless'],['firefox Ubuntu 22.04 122 selenium headless']
"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",2,NaN,NaN
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",32,10.0,16.0
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",12,36.0,30.0



('subresourceloadingCORP_img', 'direct')


,"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
error,1.0,NaN
load,NaN,1.0



('subresourceloadingCORP_object', 'direct')


,"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
error,32.0,NaN
load,NaN,32.0


In [188]:
# Simmilarity between browsers and versions!

# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display the results
    # if show_all:
        # display(results_df)
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("resp_type"):
    display_overlap(group, show_all=True, name=name)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

All


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,71,72,72,72,1057,1055
chrome Ubuntu 22.04 120 selenium headless-new,71,0,1,1,1,1015,1013
chrome Ubuntu 22.04 121 selenium headless-new,72,1,0,0,0,1016,1014
chrome Ubuntu 22.04 122 selenium headless-new,72,1,0,0,0,1016,1014
edge Ubuntu 22.04 121 selenium headless-new,72,1,0,0,0,1016,1014
firefox Ubuntu 22.04 121 selenium headless,1057,1015,1016,1016,1016,0,6
firefox Ubuntu 22.04 122 selenium headless,1055,1013,1014,1014,1014,6,0


basic


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,71,72,72,72,1057,1055
chrome Ubuntu 22.04 120 selenium headless-new,71,0,1,1,1,1015,1013
chrome Ubuntu 22.04 121 selenium headless-new,72,1,0,0,0,1016,1014
chrome Ubuntu 22.04 122 selenium headless-new,72,1,0,0,0,1016,1014
edge Ubuntu 22.04 121 selenium headless-new,72,1,0,0,0,1016,1014
firefox Ubuntu 22.04 121 selenium headless,1057,1015,1016,1016,1016,0,6
firefox Ubuntu 22.04 122 selenium headless,1055,1013,1014,1014,1014,6,0


accesswindow_direct


...

,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,firefox Ubuntu 22.04 121 selenium headless
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,32
chrome Ubuntu 22.04 120 selenium headless-new,0,32
chrome Ubuntu 22.04 121 selenium headless-new,0,32
chrome Ubuntu 22.04 122 selenium headless-new,0,32
edge Ubuntu 22.04 121 selenium headless-new,0,32
firefox Ubuntu 22.04 121 selenium headless,32,0
firefox Ubuntu 22.04 122 selenium headless,32,0


upgradeHSTS_direct


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0
chrome Ubuntu 22.04 120 selenium headless-new,0
chrome Ubuntu 22.04 121 selenium headless-new,0
chrome Ubuntu 22.04 122 selenium headless-new,0
edge Ubuntu 22.04 121 selenium headless-new,0
firefox Ubuntu 22.04 121 selenium headless,0
firefox Ubuntu 22.04 122 selenium headless,0


upgradeHSTS_subdomain


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0
chrome Ubuntu 22.04 120 selenium headless-new,0
chrome Ubuntu 22.04 121 selenium headless-new,0
chrome Ubuntu 22.04 122 selenium headless-new,0
edge Ubuntu 22.04 121 selenium headless-new,0
firefox Ubuntu 22.04 121 selenium headless,0
firefox Ubuntu 22.04 122 selenium headless,0


In [187]:
# How many rows exist for tree creation
df[["test_name", "label", "relation_info"]].value_counts()

test_name                      label       relation_info 
fullscreen_iframe              PP          child_allow       2688
                                           child             2688
                                           direct            2688
                                           child_sandbox     2688
framing_iframe                 CSP-FA      nested            1904
                                           sandbox           1904
framing_object                 CSP-FA      sandbox           1904
                                           direct            1904
                                           nested            1904
framing_iframe                 CSP-FA      direct            1904
framing_embed                  CSP-FA      sandbox           1904
script_execution_iframe        CSP-SCRIPT  direct            1904
                                           sandbox           1904
imgloading_iframe              CSP-IMG     direct            1904
framing_embed     

In [227]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_id"], observed=True)["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers", len(tree_df))
base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,8 days 1 hour 29 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,100.0 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Remove tests that are the same in all browsers 67632
('accesswindow_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,8 days 1 hour 29 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,100.0 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('accesswindow_direct', 'direct'), datapoints: 572
('fetch_GET', 'credentials')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


...

Create tree: ('subresourceloadingCORP_object', 'direct'), datapoints: 1552
('upgradeHSTS_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,8 days 1 hour 30 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_direct', 'direct'), datapoints: 2450
('upgradeHSTS_subdomain', 'subdomain')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,8 days 1 hour 30 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_subdomain', 'subdomain'), datapoints: 808


# Manual analysis

In [ ]:
# Chromium-based treats host-source as case sensitive even though it should not? (URLs and CSP is case-insensitive!)
# Play around: https://observer.sectec.rocks/opg/iframe/?url=https://echo.sectec.rocks/echo/?content-security-policy=img-src%20HTTPS://echo.sectec.rocks&ecocnt_css=%3Cimg%20src=https://echo.sectec.rocks/%3E&content-type=text/html
# Issue is in the scheme match which should be case-insensitive but is in Chromium-based browsers https://w3c.github.io/webappsec-csp/#match-schemes
# host-part matching was fixed here https://issues.chromium.org/issues/41412675, seems like they forgot fixing scheme-sensitivity 
df.loc[df["raw_header"].str.contains("img-src HTTPS://")][["clean_url", "raw_header", "outcome_str", "browser"]].value_counts().to_frame()

In [193]:
bf = df.set_index(["test_id", "browser"])["outcome_str"].unstack().astype("str")

def com_browsers(df, b1, b2):
    bf["cr"] = bf[b1] != bf[b2]
    with pd.option_context("display.max_colwidth", None):
        display(bf.loc[bf["cr"]][[b1, b2]])

In [204]:
# Chromium 122 vs 121 vs 120 (desktop linux)
com_browsers(bf, "chrome Ubuntu 22.04 120 selenium headless-new", "chrome Ubuntu 22.04 121 selenium headless-new")
com_browsers(bf, "chrome Ubuntu 22.04 120 selenium headless-new", "chrome Ubuntu 22.04 122 selenium headless-new")
df.loc[df["response_id"] == 196]["raw_header"].iloc[0]

browser,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new
test_id,,
referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic,document.referrer: full_url,document.referrer: http://sub.headers.websec.saarland/


browser,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new
test_id,,
referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic,document.referrer: full_url,document.referrer: http://sub.headers.websec.saarland/


"[['Referrer-Policy', 'origin-when-cross-origin']]"

In [206]:
# Firefox 120 vs 121 (desktop linux)
com_browsers(bf, "firefox Ubuntu 22.04 121 selenium headless", "firefox Ubuntu 22.04 122 selenium headless")
df.loc[df["response_id"] == 65]["raw_header"].iloc[0]

browser,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless
test_id,,
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_headers.websec.saarland_61_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_61_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_61_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.websec.saarland_65_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_sub.headers.websec.saarland_65_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_sub.sub.headers.websec.saarland_65_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"


'[]'

In [209]:
# Brave 121 vs Chrome 121
with pd.option_context("display.max_rows", 72):
    com_browsers(bf, "brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new", "chrome Ubuntu 22.04 121 selenium headless-new")

browser,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new
test_id,,
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_192_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_198_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_200_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.websec.saarland_192_basic,document.referrer: http://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.websec.saarland_198_basic,document.referrer: http://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.websec.saarland_200_basic,document.referrer: http://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_192_basic,document.referrer: http://sub.sub.headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_198_basic,document.referrer: http://sub.sub.headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_200_basic,document.referrer: http://sub.sub.headers.websec.saarland/,document.referrer: full_url


In [212]:
df.loc[df["test_id"] == "referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic"]["clean_url"].iloc[0]

'http://sub.headers.websec.saarland/_hp/tests/referrer-access-rp.sub.html?resp_type=basic&browser_id=1&label=RP&first_id=196&last_id=196&scheme=http'

# Test improved repeat mode

In [113]:
def create_test_id(row):
    return f'{row["test_name"]}_{row["relation_info"]}_{row["org_scheme"]}_{row["org_host"]}_{row["resp_scheme"]}_{row["resp_host"]}_{row["response_id"]}_{row["resp_type"]}'

df["browser_id"] = df["browser_id"].astype("category")
# Takes a while (500s+) (might be faster to already do it with postgres but not too important)
df["test_id"] = df.apply(create_test_id, axis=1)
df["test_id"] = df["test_id"].astype("category")

/tmp/ipykernel_802996/2089415988.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["browser_id"] = df["browser_id"].astype("category")
/tmp/ipykernel_802996/2089415988.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["test_id"] = df.apply(create_test_id, axis=1)
/tmp/ipykernel_802996/2089415988.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [144]:
# Luckily this is now instant
test_counts = df.groupby(["test_id"], observed=True)["browser_id"].value_counts()

In [156]:
tests_to_repeat = test_counts.loc[test_counts < 5].reset_index()
tests_to_repeat = tests_to_repeat.loc[tests_to_repeat["browser_id"] != 12]
tests_to_repeat.head()

,test_id,browser_id,count
4147,accesswindow_direct_direct_https_sub.headers.w...,46,3
4198,accesswindow_direct_direct_https_sub.headers.w...,46,3
4234,accesswindow_direct_direct_https_sub.headers.w...,46,3
4532,accesswindow_direct_direct_https_sub.headers.w...,46,3
4755,accesswindow_direct_direct_https_sub.headers.w...,46,3


In [157]:
tests_to_repeat[["browser_id", "count"]].value_counts()

browser_id  count
46          3        19879
            2            4
Name: count, dtype: int64

In [164]:
rep = tests_to_repeat.merge(df.drop_duplicates(subset=["test_id"]), on=["test_id"], how="left")

In [166]:
rep["full_url"].iloc[0]

'https://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?timeout=5&resp_type=parsing&browser_id=42&label=COOP&first_id=22971&last_id=22971&scheme=https'

In [32]:
tsv = pd.read_csv("browser_count.tsv", sep=" ")
tsv.sort_values(by=["browser_id", "count", "value_count"])

,browser_id,count,value_count
9,2,0,1281
3,2,1,133057
4,2,2,32561
6,2,3,1497
12,2,4,764
15,3,0,268
24,3,1,24
19,3,2,126
14,3,3,360
11,3,4,863
